In [ ]:
import numpy as np
import pandas as pd

# importieren der daten

df = pd.read_csv("ESS8e02.1_F1.csv", sep=",")
print(df)

In [ ]:
# beschriftung der daten
df["gndr"] = pd.cut(df["gndr"], [0,1,2,9], labels=["Male", "Female", "No Answer"])

In [ ]:
# datensatz aus deutschland
df_de = df.loc[df.cntry == "DE"]
print(df_de)

In [ ]:
# datensatz aus deutschland und italien
df_de_it = df.loc[df.cntry.isin(["DE", "IT"])]
print(df_de_it)

In [ ]:
# nominalskala: x^2


In [ ]:
# ordinalskala: korrelation

In [ ]:
# intervall: korrelation

In [ ]:
# ordinalskala: mann-whitney-u test

In [ ]:
# intervall: t-test

In [ ]:
# aufbereitung der daten

In [ ]:
# grafische veranschaulichung

In [ ]:
# hypothesentest a:

In [ ]:
# hypothesentest b:

In [ ]:
# hypothesentest c:

In [ ]:
# hypothesentest d:

In [ ]:
# hypothesentest e:

In [ ]:
# interpretation der ergebnisse: 